In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from PIL import Image

In [2]:
with open('full_input_original_human_labels.json', 'r') as file:
    human_dict = json.load(file)
    
with open('full_input_original_model_labels.json', 'r') as file:
    model_dict = json.load(file)
    
spot_keys = list(human_dict[list(human_dict.keys())[0]].keys())

In [3]:
def build_df(method_keys):
    totals = {}
    errors = {}
    false_positives = {}
    false_negatives = {}
    for spot_key in spot_keys:
        totals[spot_key] = 0
        errors[spot_key] = 0
        false_positives[spot_key] = 0
        false_negatives[spot_key] = 0

    for key in method_keys:
        for spot_key in spot_keys:
            if(human_dict[key][spot_key]):
                totals[spot_key] = totals[spot_key] + 1
            if(human_dict[key][spot_key] != model_dict[key][spot_key]):
                errors[spot_key] = errors[spot_key] + 1
                if(not human_dict[key][spot_key]):
                    false_positives[spot_key] = false_positives[spot_key] + 1
                else:
                    false_negatives[spot_key] = false_negatives[spot_key] + 1

    # Create a structured numpy array
    dtype = [('Times Occupied', int), ('Errors', int), ('False Positives', int), ('False Negatives', int)]
    values = [(totals[key], errors[key], false_positives[key], false_negatives[key]) for key in totals.keys()]
    structured_np = np.array(values, dtype=dtype)
    df = pd.DataFrame(structured_np)

    # Calculate True Negatives
    df["True Negatives"] = len(method_keys) - df["False Negatives"]

    # Calculate Accuracy
    df["Accuracy"] = (df["Times Occupied"] + df["True Negatives"]) / (df["Times Occupied"] + df["True Negatives"] + df["False Positives"] + df["False Negatives"])

    # Calculate Precision
    df["Precision"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Positives"])

    # Calculate Recall
    df["Recall"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Negatives"])

    # Calculate F1 Score
    df["F1 Score"] = 2 * (df["Precision"] * df["Recall"]) / (df["Precision"] + df["Recall"])
    df.index = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']
    df = df.round(2)
    return df

In [4]:
def build_df_dict(method_keys, meth_dict):
    totals = {}
    errors = {}
    false_positives = {}
    false_negatives = {}
    for spot_key in spot_keys:
        totals[spot_key] = 0
        errors[spot_key] = 0
        false_positives[spot_key] = 0
        false_negatives[spot_key] = 0

    for key in method_keys:
        for spot_key in spot_keys:
            if(human_dict[key][spot_key]):
                totals[spot_key] = totals[spot_key] + 1
            if(human_dict[key][spot_key] != meth_dict[key][spot_key]):
                errors[spot_key] = errors[spot_key] + 1
                if(not human_dict[key][spot_key]):
                    false_positives[spot_key] = false_positives[spot_key] + 1
                else:
                    false_negatives[spot_key] = false_negatives[spot_key] + 1

    # Create a structured numpy array
    dtype = [('Times Occupied', int), ('Errors', int), ('False Positives', int), ('False Negatives', int)]
    values = [(totals[key], errors[key], false_positives[key], false_negatives[key]) for key in totals.keys()]
    structured_np = np.array(values, dtype=dtype)
    df = pd.DataFrame(structured_np)

    # Calculate True Negatives
    df["True Negatives"] = len(method_keys) - df["False Negatives"]

    # Calculate Accuracy
    df["Accuracy"] = (df["Times Occupied"] + df["True Negatives"]) / (df["Times Occupied"] + df["True Negatives"] + df["False Positives"] + df["False Negatives"])

    # Calculate Precision
    df["Precision"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Positives"])

    # Calculate Recall
    df["Recall"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Negatives"])

    # Calculate F1 Score
    df["F1 Score"] = 2 * (df["Precision"] * df["Recall"]) / (df["Precision"] + df["Recall"])
    df.index = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']
    df = df.round(2)
    return df

In [5]:
interval = 3
count = 0
total = 0
ordered_keys = list(human_dict.keys())
ordered_keys.sort()
training_keys = []
testing_keys = []
# for key in ordered_keys:
#     print(str(count) + ' ' + key[12:24], end='')
#     if(human_dict[key][spot_key] != model_dict[key][spot_key]):
#         print('-xxxxxx', end='')
#     print('',end='\n')
#     count = count + 1

# Model doesn't make a mistake for 
training_keys = ordered_keys[0:171] 
testing_keys = ordered_keys[171:]

In [6]:
build_df(training_keys)

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,33,1,1,0,171,1.00,0.97,1.00,0.99
A2,37,0,0,0,171,1.00,1.00,1.00,1.00
A3,28,1,0,1,170,0.99,1.00,0.97,0.98
B1,22,0,0,0,171,1.00,1.00,1.00,1.00
B2,27,0,0,0,171,1.00,1.00,1.00,1.00
B3,69,2,2,0,171,0.99,0.97,1.00,0.99
B4,71,0,0,0,171,1.00,1.00,1.00,1.00
B5,119,1,1,0,171,1.00,0.99,1.00,1.00
B6,89,0,0,0,171,1.00,1.00,1.00,1.00


In [7]:
build_df(testing_keys[:-150])

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,24,6,6,0,60,0.93,0.80,1.00,0.89
A2,26,11,11,0,60,0.89,0.70,1.00,0.83
A3,23,1,0,1,59,0.99,1.00,0.96,0.98
B1,20,3,1,2,58,0.96,0.95,0.91,0.93
B2,18,19,19,0,60,0.80,0.49,1.00,0.65
B3,29,15,15,0,60,0.86,0.66,1.00,0.79
B4,19,4,4,0,60,0.95,0.83,1.00,0.90
B5,29,15,15,0,60,0.86,0.66,1.00,0.79
B6,13,5,5,0,60,0.94,0.72,1.00,0.84


In [8]:
print(len(training_keys))
print(len(testing_keys))

171
210


In [9]:
# CNN model good at determining if car in spot, from notebook, will separate to another file eventually for organization
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        # Max pool layer
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        # Max pool layer
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)

        # Fully connected layer 2
        out = self.fc2(out)
        return out

# Originally in Model_Maker notebook, this preps cropped parking spaces for ML processing
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert images to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values in the range [-1, 1]
])

In [30]:
def get_models():
    old_models = {}
    for key in spot_keys:
        model_path = os.path.join('Archive', 'old_models', key + '.pth')
        old_models[key] = CNN()
        old_models[key].load_state_dict(torch.load(model_path)) 
        old_models[key].eval() 
    return old_models

def predict_image(to_predict_image, predicting_model_dict):
    ret = {}
    for key in spot_keys:
        dir_name = 'input'
        full_path = os.path.join(dir_name, to_predict_image)
        image = Image.open(full_path)
        spots_file_path = os.path.join('Archive', 'old_models', 'spots.json')
        with open(spots_file_path, 'r') as spots_file:
            spots_data = json.load(spots_file)
        x, x_w, y, y_h = spots_data[key]
        cropped_image = image.crop((x, y, x_w, y_h))
        input_tensor = transform(cropped_image)
        input_tensor = input_tensor.unsqueeze(0)
        with torch.no_grad():
            output = predicting_model_dict[key](input_tensor)
            _, predicted = torch.max(output, 1)

        prediction = predicted.item()
        ret[key] = False
        if prediction == 0: ret[key] = True
    return ret

def bulk_predict(keys, predicting_model_dict):
    ret = {}
    for x in keys:
        ret[x] = predict_image(x, predicting_model_dict)
    return ret

In [31]:
recent_model_results = bulk_predict(list(human_dict.keys()), get_models())

In [32]:
build_df_dict(testing_keys, recent_model_results)

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,65,25,20,5,205,0.92,0.76,0.93,0.84
A2,82,22,22,0,210,0.93,0.79,1.00,0.88
A3,85,3,2,1,209,0.99,0.98,0.99,0.98
B1,50,4,1,3,207,0.98,0.98,0.94,0.96
B2,65,19,19,0,210,0.94,0.77,1.00,0.87
B3,119,31,31,0,210,0.91,0.79,1.00,0.88
B4,79,4,4,0,210,0.99,0.95,1.00,0.98
B5,86,75,75,0,210,0.80,0.53,1.00,0.70
B6,80,52,35,17,193,0.84,0.70,0.82,0.75


In [33]:
build_df(testing_keys)

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,65,25,20,5,205,0.92,0.76,0.93,0.84
A2,82,22,22,0,210,0.93,0.79,1.00,0.88
A3,85,3,2,1,209,0.99,0.98,0.99,0.98
B1,50,4,1,3,207,0.98,0.98,0.94,0.96
B2,65,19,19,0,210,0.94,0.77,1.00,0.87
B3,119,31,31,0,210,0.91,0.79,1.00,0.88
B4,79,4,4,0,210,0.99,0.95,1.00,0.98
B5,86,75,75,0,210,0.80,0.53,1.00,0.70
B6,80,52,35,17,193,0.84,0.70,0.82,0.75
